<a href="https://colab.research.google.com/github/ttktjmt/mjlab_myosuite/blob/main/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!if [ ! -d "mjlab_myosuite" ]; then git clone -q https://github.com/ttktjmt/mjlab_myosuite.git; fi
%cd /content/mjlab_myosuite
!pip install mujoco==3.3.7 -q
!pip install "myosuite @ git+https://github.com/MyoHub/myosuite.git@mjx" -q
!pip install "mujoco-warp @ git+https://github.com/google-deepmind/mujoco_warp@486642c3fa262a989b482e0e506716d5793d61a9" -q
!pip install -e . -q

/content/mjlab_myosuite
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mjlab-myosuite (pyproject.toml) ... done


In [2]:
import os
from google.colab import userdata

try:
    os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
    os.environ['WANDB_ENTITY'] = userdata.get('WANDB_ENTITY')
except:
    os.environ['WANDB_MODE'] = 'disabled'

In [3]:
import subprocess
import sys
import re

WANDB_RUN_ID = "qjms7nx1"

process = subprocess.Popen(
  [
    "python",
    "scripts/play.py",
    "Mjlab-MyoSuite-myoElbowPose1D6MRandom-v0",
    "--wandb-run-path",
    f"{userdata.get('WANDB_ENTITY')}/mjlab_myosuite/{WANDB_RUN_ID}",
    "--num_envs",
    "4",
  ],
  stdout=subprocess.PIPE,
  stderr=subprocess.STDOUT,
  universal_newlines=True,
  bufsize=1,
)

detected_port = None

for line in process.stdout:
  print(line, end="")
  sys.stdout.flush()

  # Extract port number from viser output
  port_match = re.search(r":(\d{4})", line)
  if port_match and "viser" in line.lower():
    detected_port = int(port_match.group(1))
    print("\n" + "=" * 34)
    print(f"✅ Server is running on port {detected_port}!")
    print("=" * 34)
    break

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment myoFatiArmReachFixed-v0 already in registry.


In [4]:
from google.colab import output

port = detected_port if detected_port else 8081
output.serve_kernel_port_as_iframe(port=port, height=700)

<IPython.core.display.Javascript object>